#### TF.SnoqualmieData.ModelForcing
Plots and prepares Snoqualmie flux tower met data. This data is the QC and filled data from Snoqualmie for forcing a model.


In [1]:
# must insert this statement to render the plots within the notebook
# this is specific to the ipython notebook
%matplotlib inline

## Import statements
# netcdf/numpy/xray/stats
import numpy as np
from datetime import datetime, timedelta
import pandas as pd
import xray
from scipy.stats.stats import pearsonr

# OS interaction
import sys, pickle, os

# import subplots function for plotting
import seaborn as sns
sns.set_style("whitegrid")
import matplotlib
from matplotlib.pyplot import subplots
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.basemap import Basemap

# Solargeometry
import solargeo

/home/lapok/anaconda2/lib/python2.7/site-packages/pandas/computation/__init__.py:19: UserWarning: The installed version of numexpr 2.4.4 is not supported in pandas and will be not be used

  UserWarning)
/home/lapok/anaconda2/lib/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [2]:
#### Directory Lists
# Unix
if 'linux' in sys.platform:
    dir_pre = '/home/lapok/gdrive/'
# Mac
elif 'darwin' in sys.platform:
    dir_pre = '/Users/karllapo/gdrive/'

dirData = dir_pre+'GroundObs/Snoqualmie/Raw/SNQ_Data/'                # Unprocessed Snoqualmie data
dirProj = dir_pre+'SnowHydrology/proj/TurbulenceFeedbacks/'           # Project directory
dirPrint = dir_pre+'SnowHydrology/proj/TurbulenceFeedbacks/Graphics'  # Directory for printing figures
dirOut = dir_pre+'SnowHydrology/proj/TurbulenceFeedbacks/data/SNQ'    # Processed data directory

In [3]:
def calc_RH(p,q,T):
    RH = np.empty_like(T)
    if np.nanmin(T) < 200:
        T = T + 273.16
    T_0 = 273.16

    RH = .263*p*q* (np.exp( (17.67*(T-T_0)) / (T-29.65) ))**(-1)
    RH[RH > 100] = 100
    return(RH)

def calc_Tdew(T,RH):
    ## CHECKS
    if np.nanmax(T) > 100 or np.nanmin(T) > 40:
        raise ValueError('Air temperature must be in Celsius')

    if np.nanmax(RH) > 1 or np.nanmin(RH) < 0:
        RH = RH/100
        if np.nanmax(RH) > 1 or np.nanmin(RH) < 0:
            raise ValueError('Relative humidity must be a fraction on [0,1]')

    if not np.size(RH) == np.size(T):
        raise ValueError('Relative humidity and air temperature must have the same number of elements')

    ## ALGORITHM
    # When are we calculating with respect to frost or water?
    frost_ind = np.flatnonzero(T <= 0)
    water_ind = np.flatnonzero(T > 0)
    # Frost coefficients 
    b_frost = 22.587
    c_frost = 273.86
    # Water coefficients
    b_water = 17.625
    c_water = 243.03

    # Pre-allocate
    Tdew = np.empty_like(T)
    ## Calculate
    Tdew[frost_ind] = MagnusTetens(T[frost_ind],RH[frost_ind],b_frost,c_frost)
    Tdew[water_ind] = MagnusTetens(T[water_ind],RH[water_ind],b_water,c_water)

    return(Tdew)

## SUB-FUNCTION for actual expression
def MagnusTetens(T,RH,b,c):
    dew = (c*( np.log(RH) + (b * T)/(c + T) )) / ( b - np.log(RH) - (b * T)/(c + T) )
    return(dew)

In [4]:
##### Flux tower data
def parse(y, mo, dy, hr, mn):
    s = '00'
    date_str = y+':'+mo+':'+dy+'_'+hr+':'+mn+':'+s
    dt = datetime.strptime(date_str,"%Y:%m:%d_%H:%M:%S")
    return dt

# pandas read (Dataframe)
os.chdir(dirData+'Filled_Forcing')
datafile = 'Met_Forcing_SNQ_10_1_2012_5_11_2015.txt'
data = pd.read_csv(datafile,sep=r"\s*",header=None,\
                   parse_dates={'Datetime' : [0,1,2,3,4]},date_parser=parse,index_col='Datetime')
# Format
data.columns = ['seconds','precip','SWdwn','LWdwn','Tair','WIND','Press','QS']
data.drop('seconds', axis=1, inplace=True)
# RH
data['RH'] = calc_RH(data.Press.values,data.QS.values,data.Tair.values)
# Dew point temperature
data['Tdew'] = calc_Tdew(data.Tair.values-273.16,data.RH.values)

# Convert to xray Dataset
ds = xray.Dataset.from_dataframe(data)
ds = ds.rename({'Datetime': 'time'})
os.chdir(dirOut)
ds.to_netcdf('SnoqualmieObs.ModelForcing.wy2013_wy2015.nc')

print(ds)

<xray.Dataset>
Dimensions:  (time: 45741)
Coordinates:
  * time     (time) datetime64[ns] 2012-10-01 2012-10-01T00:30:00 ...
Data variables:
    precip   (time) float64 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 ...
    SWdwn    (time) float64 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 ...
    LWdwn    (time) float64 286.9 286.9 286.5 286.5 286.5 286.5 286.5 286.5 ...
    Tair     (time) float64 279.0 279.0 279.0 279.2 279.4 279.4 279.1 278.9 ...
    WIND     (time) float64 0.068 0.126 0.06 0.126 0.045 0.099 0.199 0.116 ...
    Press    (time) float64 9.088e+04 9.086e+04 9.085e+04 9.085e+04 ...
    QS       (time) float64 0.005277 0.005341 0.005394 0.005419 0.005373 ...
    RH       (time) float64 83.35 84.34 85.05 84.36 82.46 83.76 84.16 84.74 ...
    Tdew     (time) float64 3.24 3.407 3.546 3.611 3.49 3.661 3.512 3.409 ...


/home/lapok/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:11: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators; you can avoid this warning by specifying engine='python'.
